## Using PMT scores to classify marginalised households at risk of not realizing UHC. Context of distance to facility

According to World Health Organisation, UHC means that all individuals and communities receive the health services they need without suffering financial consequences. It includes the full spectrum of essential health services, from health promotion to prevention, treatment, rehabilitation, and palliative treatment across the life course. Financial consequences is attributed to factors like distance to health facility, payment for health care services. In our context, we will seek to understand how financially vulnerable are the following categories of households when seeking health services. We consider the kenya policy that all households must be within 5 kilometers of radius to their nearest health facility.


In [64]:
import psycopg2
import pandas as pd
import datetime

In [65]:
host = "Enter host name here"
username = 'Enter username here'
password = 'Enter password here'

In [66]:
def get_db_connection(host,username,password):
    conn = psycopg2.connect(
        host= host,
        database= "mjalidataexploration",
        user= username,
        password= password)
    return conn

In [67]:
conn = get_db_connection(host,username,password)
cur = conn.cursor()

### Selecting and cleaning the proxies to use

#### 1. Select datapoints from the National_safety_programme_net table

1. hhid - Household ID
2. Villagenumber
3. membernumber
4. maritalstatus
5. urbanconstantfigure
6. thehigheststdorformreached
7. mainlydoingduringlastsevendays
8. ruralconstantfigure
9. nairobiconstantfigure
10. weight_status

In [68]:
%%time
cur.execute('SELECT hhid, membernumber, maritalstatus, thehigheststdorformreached, villagenumber, urbanconstantfigure, ruralconstantfigure, nairobiurbanconstantfigure, mainlydoingduringlastsevendays, weight_status from national_safety_programme_net')
df_nspn = cur.fetchall()
df_nspn = pd.DataFrame(df_nspn)
df_nspn.columns = ['hh_id', 'member_number','marital_status','max_education','village_number','urban_constant_figure','rural_constant_figure','nairobi_constant_figure','past_week_activity','weight_status']

CPU times: total: 4.66 s
Wall time: 2min 13s


###### check for marital status

In [6]:
%%time
# Marital status data check.
def check_unknown(x):
    if((x=='--select--')| (x=='Dont Know')):
        return 'Unknown'
    else:
        return x
    
df_nspn['marital_status'] = df_nspn['marital_status'].apply(check_unknown)

CPU times: total: 203 ms
Wall time: 231 ms


###### check for education levels

In [7]:
%%time

# Education level data check
display(df_nspn['max_education'].value_counts())


def check_education(x):
    if((x=='') | (x=='Yes')|(x=='No')):
        return 'Unknown'
    else:
        return x
    
df_nspn['max_education'] = df_nspn['max_education'].apply(check_education)

Pre-Primary    385330
Secondary      106440
                37329
University       7876
No                  2
Yes                 1
Name: max_education, dtype: int64

CPU times: total: 297 ms
Wall time: 290 ms


###### Check for weighted status

In [8]:
%%time

# Weight-status level data check
display(df_nspn['weight_status'].value_counts())


def check_weight(x):
    if(x==''):
        return 'Unknown'
    else:
        return x
    
df_nspn['weight_status'] = df_nspn['weight_status'].apply(check_weight)

Rural            429867
Urban            102023
                   4565
Nairobi Urban       522
Name: weight_status, dtype: int64

CPU times: total: 203 ms
Wall time: 199 ms


In [9]:
df_nspn['weight_status'].value_counts()

Rural            429867
Urban            102023
Unknown            4565
Nairobi Urban       522
Name: weight_status, dtype: int64

###### Check data for employment status

In [10]:
%%time

def check_employment(x):
    if(x in ['Worked own or at family business or at family agriculture','Apprentice/Intern',
              'Volunteer','Worked for pay','On leave','Sick leave']):
        return 'Employed'
    
    else:
        if(x=='--select--'):
            return 'Unknown'
        
        else:
            return 'Not Employed'
    
df_nspn['employment'] = df_nspn['past_week_activity'].apply(check_employment)

CPU times: total: 156 ms
Wall time: 161 ms


In [11]:
df_nspn['employment'].value_counts()

Not Employed    298868
Employed        238061
Unknown             49
Name: employment, dtype: int64

In [12]:
df_nspn.head()


,hh_id,member_number,marital_status,max_education,village_number,urban_constant_figure,rural_constant_figure,nairobi_constant_figure,past_week_activity,weight_status,employment
0,NaN,23072830180214405131R9FRWMRSZ,Never married,University,1704,0.00000,0.88258,0.0,No work available,Rural,Not Employed
1,NaN,12071543797886813671FLE701OFZFYXJ,Married mono-gamous,Secondary,2721,0.00000,9.10998,0.0,Worked own or at family business or at family ...,Rural,Employed
2,NaN,1207154379783012542ZH39PK8KS58BAN,Married mono-gamous,Secondary,2721,0.00000,9.17028,0.0,Worked own or at family business or at family ...,Rural,Employed
3,179855.0,607333333331702253TRO60ZI42GYQLC,Widowed,University,3102,9.36318,0.00000,0.0,Worked own or at family business or at family ...,Urban,Employed
4,115853.0,3707105645499723889,Married mono-gamous,Pre-Primary,1812,0.00000,8.56175,0.0,Incapacitated,Rural,Not Employed


In [13]:
df_nspn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536978 entries, 0 to 536977
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   hh_id                    519749 non-null  float64
 1   member_number            536978 non-null  object 
 2   marital_status           536978 non-null  object 
 3   max_education            536978 non-null  object 
 4   village_number           536978 non-null  int64  
 5   urban_constant_figure    536978 non-null  float64
 6   rural_constant_figure    536978 non-null  float64
 7   nairobi_constant_figure  536978 non-null  float64
 8   past_week_activity       536978 non-null  object 
 9   weight_status            536977 non-null  object 
 10  employment               536978 non-null  object 
dtypes: float64(4), int64(1), object(6)
memory usage: 45.1+ MB


In [14]:
df_nspn.shape

(536978, 11)

In [15]:
# check for missing values
df_nspn.isnull().sum()

hh_id                      17229
member_number                  0
marital_status                 0
max_education                  0
village_number                 0
urban_constant_figure          0
rural_constant_figure          0
nairobi_constant_figure        0
past_week_activity             0
weight_status                  1
employment                     0
dtype: int64

In [16]:
# drop the missing values in df_nspn['hh_id'] and df_nspn['weight_status']
	
df_nspn.dropna(inplace=True)

In [17]:
df_nspn.isnull().sum()

hh_id                      0
member_number              0
marital_status             0
max_education              0
village_number             0
urban_constant_figure      0
rural_constant_figure      0
nairobi_constant_figure    0
past_week_activity         0
weight_status              0
employment                 0
dtype: int64

In [18]:
df_nspn.shape

(519748, 11)

In [19]:
# read the data
df_nspn.head()

,hh_id,member_number,marital_status,max_education,village_number,urban_constant_figure,rural_constant_figure,nairobi_constant_figure,past_week_activity,weight_status,employment
3,179855.0,607333333331702253TRO60ZI42GYQLC,Widowed,University,3102,9.36318,0.00000,0.0000,Worked own or at family business or at family ...,Urban,Employed
4,115853.0,3707105645499723889,Married mono-gamous,Pre-Primary,1812,0.00000,8.56175,0.0000,Incapacitated,Rural,Not Employed
5,230936.0,4707333333331987152GGZJIRE5PSQ03F,Never married,University,1485,0.00000,0.00000,10.3922,Apprentice/Intern,Nairobi Urban,Employed
6,225780.0,120715437978635254252PGQWN6UFQJFD,Married mono-gamous,Secondary,2721,0.00000,8.99025,0.0000,Worked own or at family business or at family ...,Rural,Employed
7,228411.0,4707333333339039019ACGWJF98VC92O1,Never married,Pre-Primary,1485,0.00000,0.00000,9.9264,Seeking work,Nairobi Urban,Not Employed


#### 2. Select and clean datapoints from the memberindicators table

1. hhid - Household ID
2. membernumber
3. disability
4. specifydisablity
3. gender
4. birthdate

In [20]:
%%time
pd.read_sql('''
SELECT hhid as hh_id,
    membernumber as member_number, 
    gender, 
    birthdate, 
    relationship, 
    disability,
    specifydisability as disability_type
from memberindicators

''',conn)

C:\Users\jochoniah.nzomo\.conda\envs\DataSciences\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: total: 8.97 s
Wall time: 3min 5s


,hh_id,member_number,gender,birthdate,relationship,disability,disability_type
0,5,4307007523383922910,Male,1981-08-01,Household Head,No,None
1,20,4307123979836020170,Female,2000-09-02,Household Head,No,None
2,60,3707233334882450970,Male,1971-06-15,Household Head,No,None
3,60,3707233334886591287,Female,1989-06-15,Spouse,No,None
4,66,3707233334885863185,Female,1989-10-05,Household Head,Yes,None
...,...,...,...,...,...,...,...
1887423,814086,12072870365757076576TPPMKMIEP95BM,Female,1996-11-30,Spouse,No,--select--
1887424,814086,12072870365774366149TRS47L96ARHYN,Female,2015-11-30,Child (Birth),No,--select--
1887425,814088,12072870365789252771UTTEV9BMU0SHA,Male,1975-11-30,Household Head,No,--select--
1887426,814088,12072870365719209632XZZSM3LM5SD51,Female,1977-11-30,Spouse,No,--select--


In [21]:
%%time
cur.execute('SELECT hhid, membernumber, gender, birthdate, relationship, disability,specifydisability from memberindicators')
df_MI = cur.fetchall()
df_MI = pd.DataFrame(df_MI)
df_MI.columns = ['hh_id', 'member_number','gender','birthdate', 'relationship', 'disability', 'disability_type']

CPU times: total: 11.5 s
Wall time: 3min 31s


In [28]:
df_MI.head()

,hh_id,member_number,gender,birthdate,relationship,disability,disability_type
0,5,4307007523383922910,Male,1981-08-01,Household Head,No,None
1,20,4307123979836020170,Female,2000-09-02,Household Head,No,None
2,60,3707233334882450970,Male,1971-06-15,Household Head,No,None
3,60,3707233334886591287,Female,1989-06-15,Spouse,No,None
4,66,3707233334885863185,Female,1989-10-05,Household Head,Yes,None


###### check household members relationships

In [30]:
# Replace --select-- with 'Unknown'
df_MI.loc[df_MI['relationship'] == '--select--', 'relationship'] = 'Unknown'
df_MI['relationship'].value_counts()

Child (Birth)           795378
Household Head          685473
Spouse                  251741
Grandchild              111799
Others                   21769
Brother / Sister         11401
Child (Registration)      9775
Unknown                     92
Name: relationship, dtype: int64

###### check gender disagregations

In [31]:
df_MI['gender'].value_counts()

Female    999840
Male      887588
Name: gender, dtype: int64

###### check birthdate and calculate age of household members

In [34]:
%%time

def get_age(birthdate):
    date_now = (datetime.datetime.now())
    age_calc = (date_now-birthdate).days
    return round(age_calc/365)
    
df_MI['HH_MemberAge'] = df_MI['birthdate'].apply(get_age)

CPU times: total: 34.8 s
Wall time: 36.3 s


In [35]:
df_MI.dtypes

hh_id                       int64
member_number              object
gender                     object
birthdate          datetime64[ns]
relationship               object
disability                 object
disability_type            object
HH_MemberAge                int64
dtype: object

In [36]:
# get min and max ages
p=df_MI['HH_MemberAge'].max()
q=df_MI['HH_MemberAge'].min()


display(print(p))
display(print(q))

123


None

-73


None

In [39]:
print(df_MI.shape)
df_MI = df_MI[df_MI['HH_MemberAge']>0]
print(df_MI.shape)

(1887428, 8)
(1887367, 8)


###### check disability data

In [40]:
# df_MI['disability'].value_counts()

def check_disability(x):
    disability=x['disability']
    type_disability = x['disability_type']
    
    if((type_disability in ['Visual','Physical','Mental','Speech','Others','Self-care difficulties','Hearing']) & (disability=='No')):
        return 'Yes'
    else:
        return disability
    
df_MI['disability'] = df_MI[['disability_type','disability']].apply(check_disability,axis=1)

C:\Users\jochoniah.nzomo\AppData\Local\Temp\ipykernel_33864\880652668.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_MI['disability'] = df_MI[['disability_type','disability']].apply(check_disability,axis=1)


In [41]:
def check_disability_type(x):
    disability=x['disability']
    type_disability = x['disability_type']
    
    if((type_disability in ['Visual','Physical','Mental','Speech','Others','Self-care difficulties','Hearing']) & (disability=='Yes')):
        return type_disability
    else:
        if(disability=='No'):
            return 'None'
        else:
            return 'Unknown'
        
df_MI['disability_type'] = df_MI[['disability_type','disability']].apply(check_disability_type,axis=1)

C:\Users\jochoniah.nzomo\AppData\Local\Temp\ipykernel_33864\4287810711.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_MI['disability_type'] = df_MI[['disability_type','disability']].apply(check_disability_type,axis=1)


In [42]:
display(df_MI.columns)
display(df_MI.tail())

Index(['hh_id', 'member_number', 'gender', 'birthdate', 'relationship',
       'disability', 'disability_type', 'HH_MemberAge'],
      dtype='object')

,hh_id,member_number,gender,birthdate,relationship,disability,disability_type,HH_MemberAge
1887423,814086,12072870365757076576TPPMKMIEP95BM,Female,1996-11-30,Spouse,No,None,26
1887424,814086,12072870365774366149TRS47L96ARHYN,Female,2015-11-30,Child (Birth),No,None,7
1887425,814088,12072870365789252771UTTEV9BMU0SHA,Male,1975-11-30,Household Head,No,None,47
1887426,814088,12072870365719209632XZZSM3LM5SD51,Female,1977-11-30,Spouse,No,None,45
1887427,814088,120728703657233769357QZGE13OLF5TK,Female,2005-11-30,Child (Birth),No,None,17


#### 3. Select and clean datapoints from the household table

1. hhid - Household ID
2. villageid
3. latittude
4. longittude

In [43]:
%%time
cur.execute('SELECT hhid, villageid, latitude, longitude from households')
df_HSE = cur.fetchall()
df_HSE = pd.DataFrame(df_HSE)
df_HSE.columns = ['hh_id', 'village_id','lat','long']

CPU times: total: 2.45 s
Wall time: 28.4 s


In [52]:
x= df_HSE.shape[0]
y = df_HSE.dropna()
import numpy as np
y1 = y.replace({0.0:np.nan})
y2 = y1.dropna()
y2.shape[0]/x

0.003842041878131283

In the instance we do not have the household lat and long, we will consider using the household village id/name to calculate the distance between the village where the household is located and the facility

#### 4. Select and clean datapoints from the administrative level tables

1. hhid - Household ID
2. villageid
3. communityhealthunit
4. linfacility
5. wardname
6. Latitudes
7. longittudes
8. subcountyname
9. countyname

In [53]:
%%time

cur.execute('SELECT householdnumber,hhid,villageid from households')
df_hh = cur.fetchall()
df_hh = pd.DataFrame(df_hh)
df_hh.columns = ['hh_num','hh_id','village_id']


cur.execute('SELECT villageid,villagename,communityhealthunitid from villages')
df_village = cur.fetchall()
df_village = pd.DataFrame(df_village)
df_village.columns = ['village_id','village_name','chu_id']


cur.execute('SELECT communityhealthunitid,communityhealthunitname,linkfacility,wardid from communityhealthunits')
df_chu = cur.fetchall()
df_chu = pd.DataFrame(df_chu)
df_chu.columns = ['chu_id','chu_name','link_fac','ward_id']

cur.execute('SELECT wardid,wardname,subcountyid,latitude,longitude from wards')
df_wards = cur.fetchall()
df_wards = pd.DataFrame(df_wards)
df_wards.columns = ['ward_id','ward_name','subcounty_id','lat_ward','lon_ward']

cur.execute('SELECT subcountyid, subcountyname,countyid,latitude,longitude from subcounties')
df_subcounty = cur.fetchall()
df_subcounty = pd.DataFrame(df_subcounty)
df_subcounty.columns = ['subcounty_id','subcounty_name','county_id','lat_subcounty','lon_subcounty']

cur.execute('SELECT countyid,countyname,countryid from counties')
df_county = cur.fetchall()
df_county = pd.DataFrame(df_county)
df_county.columns = ['county_id','county_name','country_id']

cur.execute('SELECT countryid,countryname from countries')
df_country = cur.fetchall()
df_country = pd.DataFrame(df_country)
df_country.columns = ['country_id','country_name']


df_ADMN = pd.merge(df_country,df_county,on='country_id')
df_ADMN = df_ADMN[df_ADMN['country_name']=='Kenya']
df_ADMN = df_ADMN[df_ADMN['county_name'].isin(['Kilifi','Makueni','Meru','Taita-Taveta','Migori','Kakamega'])]

df_ADMN = pd.merge(df_ADMN,df_subcounty,on='county_id')
df_ADMN = pd.merge(df_ADMN,df_wards,on='subcounty_id')
df_ADMN = pd.merge(df_ADMN,df_chu,on='ward_id')
df_ADMN = pd.merge(df_ADMN,df_village,on='chu_id')
df_ADMN = pd.merge(df_ADMN,df_hh,on='village_id')

CPU times: total: 2.19 s
Wall time: 40.3 s


In [54]:
df_ADMN.head()

,country_id,country_name,county_id,county_name,subcounty_id,subcounty_name,lat_subcounty,lon_subcounty,ward_id,ward_name,lat_ward,lon_ward,chu_id,chu_name,link_fac,village_id,village_name,hh_num,hh_id
0,1,Kenya,3,Kilifi,11,Kilifi North,-3.510651,39.909327,51,Tezo,-3.576325,39.868198,1498,ZOWERANI CU,1016,7262,SOYO SOYO,30743964861AS0203FUJMERTP,435321
1,1,Kenya,3,Kilifi,11,Kilifi North,-3.510651,39.909327,51,Tezo,-3.576325,39.868198,1498,ZOWERANI CU,1016,7262,SOYO SOYO,30743964861B0EAI0EXWKWCAJ,435525
2,1,Kenya,3,Kilifi,11,Kilifi North,-3.510651,39.909327,51,Tezo,-3.576325,39.868198,1498,ZOWERANI CU,1016,7262,SOYO SOYO,30743964861ZEW4CTL54E5G4Q,435760
3,1,Kenya,3,Kilifi,11,Kilifi North,-3.510651,39.909327,51,Tezo,-3.576325,39.868198,1498,ZOWERANI CU,1016,7262,SOYO SOYO,30743964861N29JOY2288RIN8,435894
4,1,Kenya,3,Kilifi,11,Kilifi North,-3.510651,39.909327,51,Tezo,-3.576325,39.868198,1498,ZOWERANI CU,1016,7262,SOYO SOYO,30743964861AC6CWRF9JUH1SK,436229


In [55]:
df_ADMN.isnull().sum()

country_id            0
country_name          0
county_id             0
county_name           0
subcounty_id          0
subcounty_name        0
lat_subcounty         0
lon_subcounty         0
ward_id               0
ward_name             0
lat_ward          17047
lon_ward          17047
chu_id                0
chu_name              0
link_fac              0
village_id            0
village_name          0
hh_num                0
hh_id                 0
dtype: int64

Next steps

1. Merge df_ADMN, df_HSE, df_nspn using hh_id
2. Augment external facility data (name and geopoints) and combine with the final merged data
3. Improve our dataset to have categories of poverty levels based on percerntiles using the urban and rural PMT scores. PMT scores are usually ranked to indicate as the rank increases, the household becomes poorer

the categories will be:

    very poor (<=25th percerntiles)
    poor (>25th percerntiles and <50th percerntiles)
    middle (>=50th percerntiles and <75th percerntiles)
    Better off (>75th percerntiles)

In [56]:
display(df_ADMN['hh_id'].nunique())
display(df_HSE['hh_id'].nunique())
display(df_nspn['hh_id'].nunique())

718065

798794

496033

In [62]:
x = set(df_ADMN['hh_id'].unique())
y = set(df_HSE['hh_id'].unique())
z = set(df_nspn['hh_id'].unique())
len(x.intersection(z))

495043

In [ ]:
dt = df_ADMN.merge(df_HSE,on = "hh_id",how = "left");print(dt.shape,df_ADMN.shape,df_HSE.shape)
dt1 = dt.merge(df_nspn,on = "hh_id",how = "left");dt1.shape


%%time

from sqlalchemy import create_engine
 
engine = create_engine(url=r"postgresql://{0}:{1}@{2}/{3}".format(username, password, host, 'PMT_UHC'))

# attach the data frame to the sql
# with a name of the table
# as "insurance_coverage" 
# *this code replaces the existing table everytime, but can be modified to append to the existing data

dt1.to_sql('Pmt_uhc', con = engine, if_exists="replace")